# 채팅 모델을 미세 조정하는 방법

이 노트는 새로운 `gpt-3.5-turbo` 미세 조정에 대한 단계별 가이드를 제공합니다. 다양한 레시피와 각각에 대해 추출된 일반 재료 목록을 제공하는 [RecipeNLG 데이터셋](https://github.com/Glorf/recipenlg)을 사용하여 엔티티 추출을 수행하겠습니다. 이 데이터셋은 명명된 엔티티 인식(NER) 작업에 흔히 사용되는 데이터셋입니다.

다음 단계를 진행합니다:

1. **설정:** 데이터 세트를 로드하고 하나의 도메인으로 필터링하여 미세 조정합니다.
2. **데이터 준비:** 학습 및 검증 예제를 생성하고 '파일' 엔드포인트에 업로드하여 미세 조정을 위한 데이터를 준비합니다.
3. **미세 조정 :** 미세 조정된 모델을 생성합니다.
4. **추론:** 새로운 입력에 대한 추론을 위해 미세 조정된 모델을 사용합니다.

이 과정을 마치면 미세 조정된 'gpt-3.5-turbo' 모델을 훈련, 평가 및 배포할 수 있어야 합니다.

미세 조정에 대한 자세한 내용은 [문서 가이드](https://platform.openai.com/docs/guides/fine-tuning), [API 참조](https://platform.openai.com/docs/api-reference/fine-tuning) 또는 [블로그 포스트](https://openai.com/blog/gpt-3-5-turbo-fine-tuning-and-api-updates)를 참조하세요.

설정 ## 설정

In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade openai 

In [1]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


미세 조정은 특정 도메인에 집중할 때 가장 효과적입니다. 데이터 세트가 모델이 학습할 수 있을 만큼 충분히 집중되어 있으면서도 보이지 않는 예시도 놓치지 않을 만큼 충분히 일반적인지 확인하는 것이 중요합니다. 이를 염두에 두고 RecipesNLG 데이터 세트에서 www.cookbooks.com 의 문서만 포함하도록 하위 집합을 추출했습니다.

In [2]:
# Read in the dataset we'll use for this task.
# This will be the RecipesNLG dataset, which we've cleaned to only contain documents from www.cookbooks.com
recipe_df = pd.read_csv("data/cookbook_recipes_nlg_10k.csv")

recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## 데이터 준비

데이터 준비부터 시작하겠습니다. ChatCompletion` 형식으로 미세 조정할 때, 각 훈련 예제는 단순한 `메시지`의 목록입니다. 예를 들어, 항목은 다음과 같을 수 있습니다:

```
[{'role': '시스템',
  'content': '귀하는 유용한 레시피 도우미입니다. 제공된 각 레시피에서 일반 재료를 추출해야 합니다.'},

 {'role': 'user',
  'content': '제목: 노베이크 너트 쿠키\n\n재료: ["단단하게 포장된 흑설탕 1컵", "증발유 1/2컵", "바닐라 1/2작은술", "잘게 부순 견과류(피칸) 1/2컵", "버터 또는 마가린 2큰술", "한 입 크기의 잘게 썬 쌀 비스킷 3 1/2컵"]\n\n일반 재료: '},

 {'role': '어시스턴트',
  'content': '["흑설탕", "우유", "바닐라", "견과류", "버터", "한입 크기 슈레드 라이스 비스킷"]'}]
```

훈련 과정에서 이 대화는 분할되어 최종 항목은 모델이 생성할 '완료'가 되고 나머지 '메시지'는 프롬프트 역할을 합니다. 훈련 예시를 만들 때 이 점을 고려하세요. 모델이 다중 턴 대화에 대해 작동하는 경우 대화가 확장되기 시작할 때 성능이 저하되지 않도록 대표적인 예시를 제공하세요.

현재 각 트레이닝 예제에는 4096개의 토큰 제한이 있다는 점에 유의하세요. 이보다 길면 4096 토큰에서 잘립니다.


In [4]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def create_user_message(row):
    return f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["NER"]})

    return {"messages": messages}

pprint(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'Title: No-Bake Nut Cookies\n'
                          '\n'
                          'Ingredients: ["1 c. firmly packed brown sugar", '
                          '"1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 '
                          'c. broken nuts (pecans)", "2 Tbsp. butter or '
                          'margarine", "3 1/2 c. bite size shredded rice '
                          'biscuits"]\n'
                          '\n'
                          'Generic ingredients: ',
               'role': 'user'},
              {'content': '["brown sugar", "milk", "vanilla", "nuts", '
                          '"butter", "bite size shredded rice biscuits"]',
               'role': 'assistant'}]}


이제 학습 데이터로 사용할 데이터 집합의 하위 집합에 대해 이 작업을 수행해 보겠습니다. 잘 정리된 30~50개의 예제로도 시작할 수 있습니다. 훈련 집합의 크기가 증가함에 따라 성능이 선형적으로 계속 확장되는 것을 볼 수 있지만 작업 시간도 더 오래 걸립니다.

In [5]:
# use the first 100 rows of the dataset for training
training_df = recipe_df.loc[0:100]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["bee

학습 데이터 외에도 **선택적으로** 유효성 검사 데이터를 제공할 수 있으며, 이 데이터는 모델이 학습 세트에 과도하게 적합하지 않은지 확인하는 데 사용됩니다.

In [6]:
validation_df = recipe_df.loc[101:200]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

그런 다음 각 줄이 하나의 훈련 예제 대화인 `.jsonl` 파일로 데이터를 저장해야 합니다.


In [7]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [8]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

트레이닝 `.jsonl` 파일의 처음 5줄은 다음과 같습니다:

In [9]:
# print the first 5 lines of the training file
!head -n 5 tmp_recipe_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: No-Bake Nut Cookies\n\nIngredients: [\"1 c. firmly packed brown sugar\", \"1/2 c. evaporated milk\", \"1/2 tsp. vanilla\", \"1/2 c. broken nuts (pecans)\", \"2 Tbsp. butter or margarine\", \"3 1/2 c. bite size shredded rice biscuits\"]\n\nGeneric ingredients: "}, {"role": "assistant", "content": "[\"brown sugar\", \"milk\", \"vanilla\", \"nuts\", \"butter\", \"bite size shredded rice biscuits\"]"}]}
{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: Jewell Ball'S Chicken\n\nIngredients: [\"1 small jar chipped beef, cut up\", \"4 boned chicken breasts\", \"1 can cream of mushroom soup\", \"1 carton sour cream\"]\n\nGeneric ingredients: "}, {"role":

### 파일 업로드

이제 미세 조정된 모델에서 사용할 파일을 '파일' 엔드포인트에 업로드할 수 있습니다.


In [10]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-jcdvNl27iuBMZfwi4q30IIka
Validation file ID: file-O144OIHkZ1xjB32ednBmbOXP


## 미세 조정

이제 생성된 파일과 모델을 식별하기 위한 선택적 접미사를 사용하여 미세 조정 작업을 만들 수 있습니다. 응답에는 작업의 업데이트를 검색하는 데 사용할 수 있는 `id`가 포함됩니다.

참고: 파일은 먼저 시스템에서 처리되어야 하므로 `파일이 준비되지 않았습니다` 오류가 발생할 수 있습니다. 이 경우 몇 분 후에 다시 시도하세요.


In [11]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

Job ID: ftjob-9xVzrp0Oem9rWj2UFWDcFLqT
Status: created


#### 작업 상태 확인

'https://api.openai.com/v1/alpha/fine-tunes' 엔드포인트에 `GET` 요청을 보내 알파 미세 조정 작업을 나열할 수 있습니다. 이 경우 이전 단계에서 얻은 ID가 `status: succeeded`로 끝나는지 확인해야 합니다.

작업이 완료되면 `result_files`를 사용하여 유효성 검사 세트(업로드한 경우)의 결과를 샘플링하고 `fine_tuned_model` 매개 변수의 ID를 사용하여 학습된 모델을 호출할 수 있습니다.


In [12]:
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])


Job ID: ftjob-9xVzrp0Oem9rWj2UFWDcFLqT
Status: running
Trained Tokens: None


이벤트 엔드포인트로 미세 조정의 진행 상황을 추적할 수 있습니다. 미세 조정이 준비될 때까지 아래 셀을 몇 번 다시 실행할 수 있습니다.


In [13]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tune: ftjob-9xVzrp0Oem9rWj2UFWDcFLqT
Fine tuning job started
Step 10: training loss=2.41
Step 20: training loss=0.06
Step 30: training loss=0.38
Step 40: training loss=0.09
Step 50: training loss=0.19
Step 60: training loss=0.10
Step 70: training loss=0.00
Step 80: training loss=0.01
Step 90: training loss=0.72
Step 100: training loss=0.13
Step 110: training loss=0.15
Step 120: training loss=0.00
Step 130: training loss=0.47
Step 140: training loss=0.00
Step 150: training loss=0.10
Step 160: training loss=0.06
Step 170: training loss=0.03
Step 180: training loss=0.08
Step 190: training loss=0.04
Step 200: training loss=0.01
Step 210: training loss=0.03
Step 220: training loss=0.00
Step 230: training loss=0.08
Step 240: training loss=0.00
Step 250: training loss=0.00
Step 260: training loss=0.00
Step 270: training loss=0.00
Step 280: training loss=0.00
Step 290: training loss=0.00
Step 300: training loss=0.60
New fine-tuned model created: ft:gpt-3.5-turbo-0613:openai:recipe

이제 작업이 완료되었으므로 작업에서 미세 조정된 모델 ID를 얻을 수 있습니다:


In [14]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:openai:recipe-ner:7qTvyJ81


추론 ##

마지막 단계는 추론에 미세 조정된 모델을 사용하는 것입니다. 기존의 `FineTuning`과 유사하게, `model` 매개변수를 채우는 새 미세 조정 모델 이름으로 `ChatCompletions`를 호출하기만 하면 됩니다.


In [15]:
test_df = recipe_df.loc[201:300]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": create_user_message(test_row)})

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Pancakes\n'
             '\n'
             'Ingredients: ["1 c. flour", "1 tsp. soda", "1 tsp. salt", "1 '
             'Tbsp. sugar", "1 egg", "3 Tbsp. margarine, melted", "1 c. '
             'buttermilk"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


In [16]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

["flour", "soda", "salt", "sugar", "egg", "margarine", "buttermilk"]


결론 ## 결론

축하합니다, 이제 'ChatCompletion' 형식을 사용하여 자신만의 모델을 미세 조정할 준비가 되었습니다! 여러분이 만든 모델을 기대하겠습니다.
